# Import Semantic Segmentation Dataset

In this tutorial, we will import the LIACi Semantic Segmentation Dataset for
Underwater Ship Inspections, introduced in
[this](https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=9998080) paper.

The dataset contains roughly 2000 images of underwater ship hulls, together with
their corresponding annotations. The dataset contains both COCO-style
annotations (bounding boxes and segmentation polygons) and pixel-wise
annotations stored as single-channel bitmap images.

We will import three different tables from the dataset, in order to showcase
different ways of working with annotated image dat in 3LC:

1. `tlc.Table.from_coco()` to import the COCO-style bounding box annotations
   (NOTE: 3LC does not yet support segmentation polygons, support for this is
   right around the corner)
2. `tlc.Table.from_torch_dataset()` using a custom torch dataset where the mask
   images from all classes are merged into a single segmentation mask
3. `tlc.Table.from_torch_dataset()` using a custom torch dataset which returns all
   the 10 masks as separate elements

<div style="display: inline-flex; align-items: center; gap: 10px;">
        <a href="https://colab.research.google.com/github/3lc-ai/3lc-examples/blob/main/tutorials/import-sintef-dataset.ipynb"
        target="_blank"
            style="background-color: transparent; text-decoration: none; display: inline-flex; align-items: center;
            padding: 5px 10px; font-family: Arial, sans-serif;"> <img
            src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab" style="height: 30px;
            vertical-align: middle;box-shadow: none;"/>
        </a> <a href="https://github.com/3lc-ai/3lc-examples/blob/main/tutorials/import-sintef-dataset.ipynb"
            style="text-decoration: none; display: inline-flex; align-items: center; background-color: #ffffff; border:
            1px solid #d1d5da; border-radius: 8px; padding: 2px 10px; color: #333; font-family: Arial, sans-serif;">
            <svg aria-hidden="true" focusable="false" role="img" class="octicon octicon-mark-github" viewBox="0 0 16 16"
            width="20" height="20" fill="#333"
            style="display:inline-block;user-select:none;vertical-align:text-bottom;overflow:visible; margin-right:
            8px;">
                <path d="M8 0c4.42 0 8 3.58 8 8a8.013 8.013 0 0 1-5.45 7.59c-.4.08-.55-.17-.55-.38 0-.27.01-1.13.01-2.2
                0-.75-.25-1.23-.54-1.48 1.78-.2 3.65-.88 3.65-3.95 0-.88-.31-1.59-.82-2.15.08-.2.36-1.02-.08-2.12 0
                0-.67-.22-2.2.82-.64-.18-1.32-.27-2-.27-.68 0-1.36.09-2 .27-1.53-1.03-2.2-.82-2.2-.82-.44 1.1-.16
                1.92-.08 2.12-.51.56-.82 1.28-.82 2.15 0 3.06 1.86 3.75 3.64 3.95-.23.2-.44.55-.51
                1.07-.46.21-1.61.55-2.33-.66-.15-.24-.6-.83-1.23-.82-.67.01-.27.38.01.53.34.19.73.9.82 1.13.16.45.68
                1.31 2.69.94 0 .67.01 1.3.01 1.49 0 .21-.15.45-.55.38A7.995 7.995 0 0 1 0 8c0-4.42 3.58-8 8-8Z"></path>
            </svg> <span style="vertical-align: middle; color: #333;">Open in GitHub</span>
        </a>
</div>

## Setup Project

In [ ]:
PROJECT_NAME = "LIACI"
DATASET_NAME = "LIACI"

INSTALL_DEPENDENCIES = False


In [ ]:
%%capture
if INSTALL_DEPENDENCIES:
    %pip --quiet install 3lc
    %pip --quiet install torch torchvision

## Imports

In [ ]:
import tlc
from torch.utils.data import Dataset
import os
from PIL import Image
import numpy as np
from colorsys import hls_to_rgb

## Prepare Dataset

The dataset is available for download from the [official website](https://data.sintef.no/product/details/dp-9e112cec-3a59-4b58-86b3-ecb1f2878c60), and must be downloaded and extracted to a local directory manually.

The dataset is stored in the following layout: 

```
LIACi_dataset_pretty
│
├── images
│   ├── image_0001.jpg
│   ├── image_0002.jpg
│   ├── image_0003.jpg
│   └── ...
│
├── masks
│   ├── anode
│   │   ├── image_0001.bmp
│   │   ├── image_0002.bmp
│   │   ├── image_0003.bmp
│   │   └── ...
│   ├── bilge_keel
│   ├── corrosion
│   ├── defect
│   ├── marine_growth
│   ├── over_board_valves
│   ├── paint_peel
│   ├── propeller
│   ├── saliency
│   ├── sea_chest_grating
│   ├── segmentation
│   └── ship_hull
│
├── coco-annotations.json
├── train_test_split.csv
...
```

In other words, there is a single binary mask for each class for each image.



In [ ]:
DATASET_ROOT = "C:/Data/LIACi_dataset_pretty"

tlc.register_url_alias("LIACI_DATASET_ROOT", DATASET_ROOT)

## Approach 1: Import COCO-style Annotations

In [ ]:
table_from_coco = tlc.Table.from_coco(
    annotations_file=f"{DATASET_ROOT}/coco-labels.json",
    image_folder=f"{DATASET_ROOT}/images",
    project_name=PROJECT_NAME,
    dataset_name=DATASET_NAME,
    table_name="coco",
)


## Approach 2: Import Merged Masks

In [ ]:
# Read out the value_map from the first table:
value_map = table_from_coco.get_value_map("bbs.bb_list.label")

# Create a new simplified value map, using a corrected label name:
simple_value_map = {map_item.internal_name: int(value) for value, map_item in value_map.items()}
# Rename the "over_board_valve" key to "over_board_valves", as the COCO category and the folder name differ:
simple_value_map["over_board_valves"] = simple_value_map["over_board_valve"]
del simple_value_map["over_board_valve"]
print(simple_value_map)


In [ ]:
# Define some helpers:

def generate_hsi_colors(num_colors=10):
    """Generate a list of distinct colors in HSI space."""
    colors = []
    saturation = 1.0
    intensity = 0.7
    hues = np.linspace(0, 1, num_colors, endpoint=False)
    for hue in hues:
        rgb = hls_to_rgb(hue, intensity, saturation)
        colors.append(rgb_to_hex(rgb))
    return colors

def rgb_to_hex(rgb):
    """Convert an RGB tuple to a hex string."""
    return '#{:02x}{:02x}{:02x}'.format(int(rgb[0] * 255), int(rgb[1] * 255), int(rgb[2] * 255))


colors = generate_hsi_colors(num_colors=10)

In [ ]:
# Add some colors to the value map:
for ind, map_element in enumerate(value_map.values()):
    map_element.display_color = colors[ind]

In [ ]:
# Define a torch Dataset returning (image, merged_mask) pairs:
class LIACIDataset(Dataset):
    def __init__(self, root, inverse_value_map):
        self.root = root
        self.inverse_value_map = inverse_value_map
        image_folder = f"{root}/images"
        self.image_files = os.listdir(image_folder)

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        image_file = self.image_files[idx]
        image_path = f"{self.root}/images/{image_file}"
        image = Image.open(image_path)
        mask = self._make_mask(image_file.replace(".jpg", ".bmp"))
        return image, mask

    def _make_mask(self, image_file) -> Image:
        # Merge all 10 binary masks into a single multiclass mask for this image
        # Create an empty array for the categorical mask, initialized to 0 (background)
        mask_shape = None
        merged_mask = None

        # Iterate over all categories
        for category, category_id in self.inverse_value_map.items():
            # Build the path to the current category mask
            category_mask_path = f"{self.root}/masks/{category}/{image_file}"

            # Open the binary mask for this category
            category_mask = Image.open(category_mask_path)

            # Convert the category mask to a numpy array
            category_mask_array = np.array(category_mask)

            # Ensure that the merged mask is initialized only once, with the correct shape
            if mask_shape is None:
                mask_shape = category_mask_array.shape
                merged_mask = np.zeros(mask_shape, dtype=np.uint8)

            # Assign the category ID to the merged mask wherever the binary mask is 1
            merged_mask[category_mask_array == 1] = category_id

        # Convert the merged mask back to a PIL Image
        categorical_mask = Image.fromarray(merged_mask)

        return categorical_mask


dataset = LIACIDataset(DATASET_ROOT, simple_value_map)


In [ ]:
merged_mask_table = tlc.Table.from_torch_dataset(
    dataset,
    (tlc.PILImage("image"), tlc.SegmentationPILImage("segmentation_map", classes=value_map)),
    project_name=PROJECT_NAME,
    dataset_name=DATASET_NAME,
    table_name="merged-masks",
)


## Approach 3: Import Separate Masks

In [ ]:
class LIACIDatasetV2(Dataset):
    def __init__(self, root, inverse_value_map):
        self.root = root
        self.inverse_value_map = inverse_value_map
        image_folder = f"{root}/images"
        self.image_files = os.listdir(image_folder)

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        image_file = self.image_files[idx]
        image_path = f"{self.root}/images/{image_file}"
        image = Image.open(image_path)

        masks = (
            Image.open(f"{self.root}/masks/{label}/{image_file.replace('.jpg', '.bmp')}")
            for label in self.inverse_value_map.keys()
        )
        return image, *masks

dataset = LIACIDatasetV2(DATASET_ROOT, simple_value_map)


In [ ]:
dataset[0]

In [ ]:
mask_structures = (
    tlc.SegmentationPILImage(
        f"{label.internal_name}_mask",
        classes={0.0: tlc.MapElement("background"), 255.0: label},
    )
    for label in enumerate(
        value_map.values(),
    )
)
structure = (tlc.PILImage("image"), *mask_structures)

separate_masks_table = tlc.Table.from_torch_dataset(
    dataset,
    structure,
    project_name=PROJECT_NAME,
    dataset_name=DATASET_NAME,
    table_name="separate-masks",
)
